<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Données" data-toc-modified-id="Données-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Données</a></span></li></ul></li><li><span><a href="#Import" data-toc-modified-id="Import-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import</a></span><ul class="toc-item"><li><span><a href="#Outils" data-toc-modified-id="Outils-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Outils</a></span></li></ul></li><li><span><a href="#S3-aws-avec-boto3" data-toc-modified-id="S3-aws-avec-boto3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>S3 aws avec boto3</a></span><ul class="toc-item"><li><span><a href="#Credentials" data-toc-modified-id="Credentials-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Credentials</a></span></li></ul></li><li><span><a href="#Session-Spark" data-toc-modified-id="Session-Spark-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Session Spark</a></span><ul class="toc-item"><li><span><a href="#Info-sur-spark-et-la-session" data-toc-modified-id="Info-sur-spark-et-la-session-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Info sur spark et la session</a></span></li></ul></li><li><span><a href="#Load-dataframe" data-toc-modified-id="Load-dataframe-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load dataframe</a></span><ul class="toc-item"><li><span><a href="#Dataframe-des-images-dans-S3" data-toc-modified-id="Dataframe-des-images-dans-S3-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Dataframe des images dans S3</a></span></li><li><span><a href="#Dataframe-des-images-en-locale" data-toc-modified-id="Dataframe-des-images-en-locale-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Dataframe des images en locale</a></span></li><li><span><a href="#Label-de-l'image" data-toc-modified-id="Label-de-l'image-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Label de l'image</a></span></li></ul></li><li><span><a href="#Reduction-avec-PCA" data-toc-modified-id="Reduction-avec-PCA-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Reduction avec PCA</a></span><ul class="toc-item"><li><span><a href="#Création-d'un-vecteur-dense" data-toc-modified-id="Création-d'un-vecteur-dense-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Création d'un vecteur dense</a></span></li><li><span><a href="#Standard-Scaler" data-toc-modified-id="Standard-Scaler-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Standard Scaler</a></span></li><li><span><a href="#PCA" data-toc-modified-id="PCA-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>PCA</a></span></li><li><span><a href="#Save-Dataframe" data-toc-modified-id="Save-Dataframe-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Save Dataframe</a></span></li></ul></li><li><span><a href="#GIT" data-toc-modified-id="GIT-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>GIT</a></span></li><li><span><a href="#Bibliography" data-toc-modified-id="Bibliography-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Bibliography</a></span></li></ul></div>

# Introduction

Dans un environnement Big Data, une première chaîne de traitement des données qui comprendra le preprocessing et une étape de réduction de dimension.
La sortie de la réduction de dimension (une matrice écrite sur un fichier CSV ou autre) est ecrite dans un espace de stockage sur le cloud.



## Données

https://www.kaggle.com/datasets/moltean/fruits?select=fruits-360_dataset


# Import


In [1]:
# pyspark
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import split, udf, col, pandas_udf, PandasUDFType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, BinaryType, DoubleType, DataType, FloatType
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import PCA, StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector


# image
from PIL import Image, ImageDraw

# Outils
import numpy as np
import pandas as pd
import sys
import os
import io
import time
from contextlib import contextmanager


import boto3


## Outils


In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))


In [3]:
s3bucket = 'ocunderscorep8'
debug_folder = 'fruits-360-debug/Training/'
full_folder = 'fruits-360/Training/'

# S3 aws avec boto3


https://us-east-1.console.aws.amazon.com/iam/home#/users$new?step=permissions&accessKey&userNames=app_p8&permissionType=policies

In [5]:
session = boto3.session.Session(aws_access_key_id="",
                                        aws_secret_access_key="")
s3_client = session.client(service_name='s3')
prefix = debug_folder #fruits-360/Training/
sub_folders = s3_client.list_objects_v2(Bucket=s3bucket, Prefix=prefix)


Mes buckets sur s3

In [6]:
s3_client.list_buckets()

{'ResponseMetadata': {'RequestId': 'AMGBASYZN52ZMYK8',
  'HostId': 'Y73KzsnxcNnvMG9V7XKSogZjk4wKBLfao5zwz6mkFXZC1VhKnnuk4997mRWtCn1D1HRFg/o9bwM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Y73KzsnxcNnvMG9V7XKSogZjk4wKBLfao5zwz6mkFXZC1VhKnnuk4997mRWtCn1D1HRFg/o9bwM=',
   'x-amz-request-id': 'AMGBASYZN52ZMYK8',
   'date': 'Sun, 20 Nov 2022 23:02:52 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aws-logs-479126531994-eu-west-3',
   'CreationDate': datetime.datetime(2022, 11, 13, 21, 40, 19, tzinfo=tzutc())},
  {'Name': 'ocunderscorep8',
   'CreationDate': datetime.datetime(2022, 11, 11, 15, 9, 24, tzinfo=tzutc())},
  {'Name': 'sagemaker-eu-west-3-479126531994',
   'CreationDate': datetime.datetime(2022, 11, 18, 17, 32, 35, tzinfo=tzutc())},
  {'Name': 'sagemaker-studio-479126531994-bb9tbgaoy2h',
   'CreationDate': datetime.datetime(2022, 11, 19, 21, 26, 35, tzinfo=tzutc(

Mes subfolder sur S3

In [7]:
sub_folders

{'ResponseMetadata': {'RequestId': 'JNZYP51RC8P1NCG2',
  'HostId': '9OQS/5qw8s6lw1EPx24Kv7CoPydBLJ2Y5sNV2aUlbpAd6ihfb4RqugjaI/slbZ2b0wNpUpolE/s=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9OQS/5qw8s6lw1EPx24Kv7CoPydBLJ2Y5sNV2aUlbpAd6ihfb4RqugjaI/slbZ2b0wNpUpolE/s=',
   'x-amz-request-id': 'JNZYP51RC8P1NCG2',
   'date': 'Sun, 20 Nov 2022 23:02:44 GMT',
   'x-amz-bucket-region': 'eu-west-3',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'IsTruncated': True,
 'Contents': [{'Key': 'fruits-360-debug/Training/Apple Braeburn/0_100.jpg',
   'LastModified': datetime.datetime(2022, 11, 13, 18, 31, 46, tzinfo=tzutc()),
   'ETag': '"2247c5e506d29555ba9bcd403cf29330"',
   'Size': 4773,
   'StorageClass': 'STANDARD'},
  {'Key': 'fruits-360-debug/Training/Apple Braeburn/100_100.jpg',
   'LastModified': datetime.datetime(2022, 11, 13, 18, 32, 5, tzinfo=tzutc()),
   'ETag': '"324fa5ff9895272562590258c73ffe18"'

In [8]:
paths =  []
for key in sub_folders["Contents"]:
    file = key["Key"]
    paths.append("s3://ocunderscorep8/" + file)
paths

['s3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/0_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/100_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/101_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/102_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/103_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/104_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/105_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/106_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/107_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/108_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/109_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/10_100.jpg',
 's3://ocunderscorep8/fruits-360-debug/Training/Apple Braeburn/110_100.jpg',
 '

## Credentials

J'utilise mes credentials root, ce qui en soi est "mal". Mais il s'agit d'un petit projet sans suite.

In [13]:
credentials = session.get_credentials()
AWS_ACCESS_KEY_ID = credentials.access_key
AWS_SECRET_ACCESS_KEY = credentials.secret_key

# Session Spark



In [20]:
spark = SparkSession.builder.appName("oc_p8").config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID).config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY).getOrCreate()
spark.sparkContext.setLogLevel("WARN")

22/11/21 00:16:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Info sur spark et la session

In [21]:

spark


In [18]:
sc = spark.sparkContext
print(f"Hadoop version = {sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

Hadoop version = 3.2.0


# Load dataframe

In [ ]:
def load_data(path):
    #chargement dataframe des images
    df_img = spark.read.format("image").option("recursiveFileLookup", True).option("dropInvalid", True).load(path)
    print(df_img.printSchema())
    return df_img

## Dataframe des images dans S3

J'ai du ajouter 3 jars qui manquanient. Voir bibliographie.



In [23]:
df2_img =load_data('s3a://ocunderscorep8/fruits-360-debug/Training/*')

22/11/21 00:16:55 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

None


In [26]:
df2_img.select("image.origin", "image.height", "image.width", "image.nChannels", "image.mode", "image.data" ).show(truncate=30)

+------------------------------+------+-----+---------+----+------------------------------+
|                        origin|height|width|nChannels|mode|                          data|
+------------------------------+------+-----+---------+----+------------------------------+
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FD FF FF FD FF FF FD FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|s3a://ocunderscorep8/fruits...|   100|  100|        3|  16|[FF FF FF FF FF FF F

## Dataframe des images en locale



In [22]:


df_img =load_data(debug_folder)


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

None


In [12]:
df_img.select("image.origin", "image.height", "image.width", "image.nChannels", "image.mode", "image.data" ).show(truncate=30)

+------------------------------+------+-----+---------+----+------------------------------+
|                        origin|height|width|nChannels|mode|                          data|
+------------------------------+------+-----+---------+----+------------------------------+
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FD FF FF FD FF FF FD FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF FF FF FF...|
|file:///Users/mcBedini/Docu...|   100|  100|        3|  16|[FF FF FF FF FF FF F

## Label de l'image

Je simplifie les labels avec un udf. Toutes les sous-catégories (ex. Pear 1, Pear 2)sont groupées dans une catégorie (ex. Pear).



In [13]:
def set_label(path):
        return path.split('/')[-2].split('_')[0]
udf_label = udf(set_label, StringType())
df_img = df_img.withColumn('Label', udf_label("image.origin"))


In [14]:
df_img.show(truncate=95)

+-----------------------------------------------------------------------------------------------+----------+
|                                                                                          image|     Label|
+-----------------------------------------------------------------------------------------------+----------+
|{file:///Users/mcBedini/Documents/Programmini/Python/oc_p8/fruits-360-debug/Training/Waterme...|Watermelon|
|{file:///Users/mcBedini/Documents/Programmini/Python/oc_p8/fruits-360-debug/Training/Waterme...|Watermelon|
|{file:///Users/mcBedini/Documents/Programmini/Python/oc_p8/fruits-360-debug/Training/Waterme...|Watermelon|
|{file:///Users/mcBedini/Documents/Programmini/Python/oc_p8/fruits-360-debug/Training/Waterme...|Watermelon|
|{file:///Users/mcBedini/Documents/Programmini/Python/oc_p8/fruits-360-debug/Training/Waterme...|Watermelon|
|{file:///Users/mcBedini/Documents/Programmini/Python/oc_p8/fruits-360-debug/Training/Waterme...|Watermelon|
|{file:///Users/mcB

Affichage de l'image de la première ligne, une pastèque

In [15]:
spark_single_img = df_img.first()
(spark_single_img.image.origin, spark_single_img.image.mode, spark_single_img.image.nChannels )

mode = 'RGBA' if (spark_single_img.image.nChannels == 4) else 'RGB' 
Image.frombytes(mode=mode, data=bytes(spark_single_img.image.data), size=[spark_single_img.image.width,spark_single_img.image.height]).show()



# Reduction avec PCA

https://stackoverflow.com/questions/73654149/pyspark-image-dimension-reduction-with-pca

## Création d'un vecteur dense

à partir de image.data. QUESTION image.data est un vecteur : https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/ml/image/ImageSchema.scala#L131-L190 With a 640 x 480 RGB image example, the first three positions of the array will contain color information(Blue, Green and Red) from the pixel at height = 0 and width = 0. The next three position will contain information from the pixel at at height = 0 and width = 1 and so on. (https://towardsdatascience.com/image-segmentation-on-apache-spark-46164dd53c73)

In [16]:
ud_img2vec = udf(lambda x : Vectors.dense(x), VectorUDT())

v_df = df_img.withColumn("data_as_vector", ud_img2vec("image.data"))

v_df.show(truncate=45)

+---------------------------------------------+----------+---------------------------------------------+
|                                        image|     Label|                               data_as_vector|
+---------------------------------------------+----------+---------------------------------------------+
|{file:///Users/mcBedini/Documents/Programm...|Watermelon|[255.0,255.0,255.0,255.0,255.0,255.0,255.0...|
|{file:///Users/mcBedini/Documents/Programm...|Watermelon|[253.0,255.0,255.0,253.0,255.0,255.0,253.0...|
|{file:///Users/mcBedini/Documents/Programm...|Watermelon|[255.0,255.0,255.0,255.0,255.0,255.0,255.0...|
|{file:///Users/mcBedini/Documents/Programm...|Watermelon|[255.0,255.0,255.0,255.0,255.0,255.0,255.0...|
|{file:///Users/mcBedini/Documents/Programm...|Watermelon|[255.0,255.0,255.0,255.0,255.0,255.0,255.0...|
|{file:///Users/mcBedini/Documents/Programm...|Watermelon|[255.0,255.0,255.0,255.0,255.0,255.0,255.0...|
|{file:///Users/mcBedini/Documents/Programm...|Watermel

In [17]:
working_df=v_df.select('image.origin', 'Label', 'data_as_vector')
working_df.show()

+--------------------+----------+--------------------+
|              origin|     Label|      data_as_vector|
+--------------------+----------+--------------------+
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[253.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,254....|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|
|file:///U

## Standard Scaler

QUESTION 0-255 probablement inutile ?

**/!\ Il faut modifer les noms de folder et remplacer les espaces par des underscore, sinon on a une erreur dans le fit.**

In [23]:
standardizer = StandardScaler(withMean=True, withStd=True, inputCol="data_as_vector", outputCol="scaledFeatures")

model_std = standardizer.fit(working_df)
df_std = model_std.transform(working_df)


In [24]:
df_std.show()

+--------------------+----------+--------------------+--------------------+
|              origin|     Label|      data_as_vector|      scaledFeatures|
+--------------------+----------+--------------------+--------------------+
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Users/mcB...|Watermelon|[253.0,255.0,255....|[-1.1689110450410...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Users/mcB...|Watermelon|[255.0,255.0,255....|[0.32574622497417...|
|file:///Use


## PCA

22/11/14 17:18:18 WARN RowMatrix: 30000 columns will require at least 7200 megabytes of memory!

In [16]:
# algorithm
pca = PCA(k=50, inputCol='scaledFeatures', outputCol='pcaFeatures')
model_pca = pca.fit(df_std)

# Transformation images
df = model_pca.transform(df_std)
    
df.show()

22/11/14 17:18:18 WARN RowMatrix: 30000 columns will require at least 7200 megabytes of memory!
22/11/14 17:19:30 ERROR Executor: Exception in task 1.0 in stage 13.0 (TID 214)]
java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:194)
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:191)
	at breeze.linalg.DenseVector$mcD$sp.<init>(DenseVector.scala:67)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.$anonfun$computeDenseVectorCovariance$1(RowMatrix.scala:160)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix$$Lambda$3045/756077709.apply(Unknown Source)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1$adapted(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce$$Lambda$133/22600334.apply(Unknown Source)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iter

Py4JJavaError: An error occurred while calling o125.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 13.0 failed 1 times, most recent failure: Lost task 3.0 in stage 13.0 (TID 216) (192.168.1.129 executor driver): java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:194)
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:191)
	at breeze.linalg.DenseVector$mcD$sp.<init>(DenseVector.scala:67)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.$anonfun$computeDenseVectorCovariance$1(RowMatrix.scala:160)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix$$Lambda$3045/756077709.apply(Unknown Source)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1$adapted(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce$$Lambda$133/22600334.apply(Unknown Source)
	at s

t$1(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1$adapted(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce$$Lambda$133/22600334.apply(Unknown Source)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.TraversableOnce.foldLeft(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce.foldLeft$(TraversableOnce.scala:160)
	at scala.collection.AbstractIterator.foldLeft(Iterator.scala:1429)
	at scala.collection.TraversableOnce.aggregate(TraversableOnce.scala:219)
	at scala.collection.TraversableOnce.aggregate$(TraversableOnce.scala:219)
	at scala.collection.AbstractIterator.aggregate(Iterator.scala:1429)
	at org.apache.spark.rdd.RDD.$anonfun$treeAggregate$3(RDD.scala:1230)
	at org.apache.spark.rdd.RDD$$Lambda$2985/699293582.apply(Unknown Source)
	at org.apache.spark.rdd.RDD.$anonfu

## Save Dataframe


In [ ]:
def save_reduction(df,path='.'):
    path=path+"/parquet_file.parquet"
    df.write.parquet(path)

save_reduction(df)

# GIT

https://github.com/cbedini/oc_p8


# Bibliography


Spark

https://medium.com/@AaronSchlegel/install-spark-on-macos-and-load-data-from-aws-s3-with-pyspark-6884bd5f0d90


Pyspark

https://livebook.manning.com/book/pyspark-in-action/chapter-3/1


Example

https://docs.databricks.com/_static/notebooks/deep-learning/deep-learning-transfer-learning-keras.html


Spark Context, Session

https://medium.com/@achilleus/spark-session-10d0d66d1d24


AWS CLI envvars

https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-envvars.html



Boto3

https://medium.com/@antonysruthy11/loading-kaggle-dataset-to-aws-s3-using-boto3-50af3e015fb2


Images processing

https://godatadriven.com/blog/real-distributed-image-processing-with-apache-spark/

https://stackoverflow.com/questions/69205589/how-preprocess-image-using-pyspark

https://www.databricks.com/blog/2018/12/10/introducing-built-in-image-data-source-in-apache-spark-2-4.html

https://spark.apache.org/docs/latest/ml-datasource.html


UDF

https://stackoverflow.com/questions/50222362/pyspark-split-the-string-column-and-join-part-of-them-to-form-new-columns


PCA

https://stackoverflow.com/questions/73654149/pyspark-image-dimension-reduction-with-pca


Upload data sur S3

https://manan.au/data-platforms/amazon-aws/loading-data-from-kaggle-to-amazon-s3-bucket/


s3a

https://stackoverflow.com/questions/58415928/spark-s3-error-java-lang-classnotfoundexception-class-org-apache-hadoop-f

https://stackoverflow.com/questions/64563127/pyspark-s3-error-java-lang-noclassdeffounderror-com-amazonaws-services-s3-mode

http://deploymentzone.com/2015/12/20/s3a-on-spark-on-aws-ec2/

https://medium.com/@erkansirin/apache-spark-aws-s3-datasource-eb65ee906e64

https://medium.com/@AaronSchlegel/install-spark-on-macos-and-load-data-from-aws-s3-with-pyspark-6884bd5f0d90

https://notadatascientist.com/running-apache-spark-and-s3-locally/
